In [1]:
import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

Checking setup...


ModuleNotFoundError: No module named 'utils.general'

In [ ]:
!touch /content/custom_data.yaml

In [39]:
import os
import pandas as pd
sample_path = "/home/muratbayrktr/Desktop/Avion/aircraft_detection/archive/dataset/dataset/0a66768d2d566e9bbbc426354db65277.csv"
labels = {"A10":0, "A400M":1,"AG600":2, "B1":3, "B2":4, "B52":5,"Be200":6, "C130":7, "C17":8, "C5":9, "E2":10, "EF2000":11, "F117":12, "F14":13, "F15":14, "F16":15, "F18":16, "F22":17, "F35":18, "F4":19, "J20":20, "JAS39":21, "MQ9":22, "Mig31":23, "Mirage2000":24, "RQ4":25, "Rafale":26, "SR71":27, "Su57":28, "Tu160":29, "Tu95":30, "U2":31, "US2":32, "V22":33, "XB70":34, "YF23":35}

def process_csv(path, label_dict):
    f = pd.read_csv(path)
    list_of_segments = []
    for i in range(len(f)):
        name = f.loc[i, "filename"] + ".txt"
        width = f.loc[i, "width"]
        try:
            clas = labels[f.loc[i, "class"]]
        except:
            print("ERROR: Class couldn't found for {}. Assigning -1.".format(name))
            clas = -1  
        height = f.loc[i, "height"]
        xmin = f.loc[i,"xmin"]
        xmax = f.loc[i,"xmax"]
        ymin = f.loc[i,"ymin"]
        ymax = f.loc[i,"ymax"]

        centerx=round(((xmin+xmax)/2)/width,4)
        centery=round(((ymin+ymax)/2)/height,4)
        new_width=round((xmax-xmin)/width,4)
        new_height=round((ymax-ymin)/height,4)

        #Write 
        list_of_segments.append((name, (str(clas), str(centerx), str(centery), str(new_width), str(new_height))))
    return list_of_segments

x = process_csv(sample_path,labels)
for i in x:
    print(i)



('0a66768d2d566e9bbbc426354db65277.txt', ('0', '0.5865', '0.6642', '0.125', '0.093'))
('0a66768d2d566e9bbbc426354db65277.txt', ('0', '0.478', '0.563', '0.132', '0.0975'))
('0a66768d2d566e9bbbc426354db65277.txt', ('0', '0.3695', '0.4708', '0.133', '0.099'))
('0a66768d2d566e9bbbc426354db65277.txt', ('0', '0.271', '0.3823', '0.138', '0.1049'))


In [48]:
x = process_csv(sample_path,labels)

def write_yolov5(val_list, path):
    f = open(path + "/" +  val_list[0][0], "a")
    for annotation in val_list:
        string = " ".join(annotation[1])
        f.write(string); f.write("\n");
    f.close()

write_yolov5(x, "/home/muratbayrktr/Desktop/Avion/aircraft_detection/archive/dataset/dataset_yolov5")

In [87]:
# python path/to/train.py --data coco128.yaml --weights yolov5s.pt --img 640
read_path = "/home/muratbayrktr/Desktop/Avion/aircraft_detection/archive/dataset/dataset"
write_path = "/home/muratbayrktr/Desktop/Avion/aircraft_detection/archive/dataset/dataset_yolov5"


from tqdm import tqdm
for file in tqdm (os.listdir(read_path), desc="Loading…", ascii=False, ncols=75):
    if os.path.isfile(os.path.join(read_path, file)) and file.endswith(".csv"):
        abs_path = read_path + "/" + file
        #print("Processing: ", abs_path)
        yolov5_vals = process_csv(path=abs_path, label_dict=labels)
        #print("Processed: \n",yolov5_vals)
        image_path = abs_path[:-3] + "jpg " + write_path + "/images/" + file[:-3] + "jpg"
        label_path = write_path + "/labels/" 
        os.system('cp ' + image_path)
        write_yolov5(val_list=yolov5_vals, path=label_path)
        #print("Writing to file {}\n".format(yolov5_vals[0][0]+".txt"))

Loading…: 100%|█████████████████████| 10586/10586 [00:32<00:00, 323.06it/s]


In [88]:
def moveFilesToVal(file_name):
  basename = os.path.basename(file_name)
  basename_no_ext = os.path.splitext(basename)[0]
  image_name = basename_no_ext + '.jpg'
  os.system('mv ./archive/dataset/dataset_yolov5/images/' + image_name + ' ./archive/dataset/dataset_yolov5/images/val' )
  os.system('mv ./archive/dataset/dataset_yolov5/labels/' + file_name + ' ./archive/dataset/dataset_yolov5/labels/val' )

def moveFilesToTrain(file_name):
  basename = os.path.basename(file_name)
  basename_no_ext = os.path.splitext(basename)[0]
  image_name = basename_no_ext + '.jpg'

  os.system('mv ./archive/dataset/dataset_yolov5/images/' + image_name + ' ./archive/dataset/dataset_yolov5/images/train' )
  os.system('mv ./archive/dataset/dataset_yolov5/labels/' + file_name + ' ./archive/dataset/dataset_yolov5/labels/train' )


def arrangeFolder():
  dirpath = "./archive/dataset/dataset_yolov5/labels"
  files = [f for f in os.listdir(dirpath) if os.path.isfile(os.path.join(dirpath, f))] # only files, not include directory
  nval = round(len(files) / 5) # use 1/4 as validation

  for i in tqdm (range(len(files)), desc="Loading…", ascii=False, ncols=75):
    #print(files[i])

    if(i < nval):
      moveFilesToVal(files[i])  
    else:
      moveFilesToTrain(files[i])

In [89]:
arrangeFolder()

Loading…: 100%|███████████████████████| 5293/5293 [00:19<00:00, 274.21it/s]


# Now the folder dataset_yolov5 is ready to be used in training

In [92]:
!ls -al ./archive/dataset/dataset_yolov5/images/train | wc -l
!ls -al ./archive/dataset/dataset_yolov5/images/val | wc -l
!ls -al ./archive/dataset/dataset_yolov5/labels/train | wc -l
!ls -al ./archive/dataset/dataset_yolov5/labels/val | wc -l

4237
1062
4237
1062


# Organize yaml file

In [109]:
#print("nc: ", len(labels.keys()))
paths = """
train: ../archive/dataset/dataset_yolov5/images/train  
val: ../archive/dataset/dataset_yolov5/images/val 

"""
labels_str = "names: ["

for label in labels.keys():
    labels_str += '\'' + label + '\''
    labels_str += ", "

labels_str = labels_str[:-2] + "]"
#print(labels_str)

final_string = paths + "nc: " + str(len(labels.keys())) + "\n" + labels_str
f = open("custom_data.yaml", "w")
f.write(final_string)
f.close()
print(final_string)


train: ../archive/dataset/dataset_yolov5/images/train  
val: ../archive/dataset/dataset_yolov5/images/val 

nc: 36
names: ['A10', 'A400M', 'AG600', 'B1', 'B2', 'B52', 'Be200', 'C130', 'C17', 'C5', 'E2', 'EF2000', 'F117', 'F14', 'F15', 'F16', 'F18', 'F22', 'F35', 'F4', 'J20', 'JAS39', 'MQ9', 'Mig31', 'Mirage2000', 'RQ4', 'Rafale', 'SR71', 'Su57', 'Tu160', 'Tu95', 'U2', 'US2', 'V22', 'XB70', 'YF23']


In [110]:
!python3 yolov5/train.py --img 640 --batch 16 --epochs 50 --data ./custom_data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=./custom_data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 1 commit. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-59-gecc2c7b torch 1.11.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1650, 3912MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor